In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v2 import MobileNetV2
from keras import Sequential
from keras.layers import Flatten, Dense

Data Loading and Augmentation

In [2]:
train_dir = r'Face Mask Dataset/Train'
test_dir = r'Face Mask Dataset/Test'
validation_dir = r'Face Mask Dataset/Validation'

In [3]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(128, 128),  # All images will be resized to 150x150
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical')

Found 10023 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


Importing Model

In [4]:
mobilenet_v2 = MobileNetV2(weights='imagenet',include_top=False,input_shape=(128,128,3))

for layer in mobilenet_v2.layers:
    layer.trainable = False

In [5]:
model = Sequential()
model.add(mobilenet_v2)
model.add(Flatten())
model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_128 (Functi (None, 4, 4, 1280)        2257984   
_________________________________________________________________
flatten (Flatten)            (None, 20480)             0         
_________________________________________________________________
dense (Dense)                (None, 2)                 40962     
Total params: 2,298,946
Trainable params: 40,962
Non-trainable params: 2,257,984
_________________________________________________________________


In [6]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics ="accuracy")

Training

In [7]:
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=len(train_generator)//32,
                              epochs=20,validation_data=validation_generator,
                              validation_steps=len(validation_generator)//32)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
9/9 [==============================] - 3s 283ms/step - loss: 1.8269 - accuracy: 0.7188
Epoch 2/20
9/9 [==============================] - 2s 219ms/step - loss: 0.5049 - accuracy: 0.9306
Epoch 3/20
9/9 [==============================] - 2s 191ms/step - loss: 0.2110 - accuracy: 0.9688
Epoch 4/20
9/9 [==============================] - 2s 230ms/step - loss: 0.2306 - accuracy: 0.9479
Epoch 5/20
9/9 [==============================] - 2s 209ms/step - loss: 0.3274 - accuracy: 0.9479
Epoch 6/20
9/9 [==============================] - 2s 204ms/step - loss: 0.2566 - accuracy: 0.9514
Epoch 7/20
9/9 [==============================] - 2s 227ms/step - loss: 0.1885 - accuracy: 0.9792
Epoch 8/20
9/9 [==============================] - 2s 213ms/step - loss: 0.1761 - accuracy: 0.9549
Epoch 9/20
9/9 [==============================] - 2s 234ms/step - loss: 0.1888 - accuracy: 0.9618
Epoch 10/20
9/9 [=========================

In [8]:
model.save('masknet_try2.h5')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = train_datagen.flow_from_directory(directory=test_dir,target_size=(128,128),class_mode='categorical',batch_size=32)

In [ ]:
model.evaluate_generator(test_generator)

In [ ]:
import cv2 
import matplotlib.pyplot as plt
import numpy as np
sample_mask_img = cv2.imread(test_dir+r'\Withmask\190.png')
sample_mask_img = cv2.resize(sample_mask_img,(128,128))
plt.figure(figsize=(1,1))
x=cv2.cvtColor(sample_mask_img, cv2.COLOR_BGR2RGB)
plt.imshow(x)
sample_mask_img = np.reshape(sample_mask_img,[1,128,128,3])
sample_mask_img = sample_mask_img/255.0

In [ ]:
model.predict(sample_mask_img)